#### 주요 클래스 IMPORT

In [ ]:
from typing import Dict, Any, Optional, Callable
from dataclasses import dataclass
from datetime import datetime
import logging

In [ ]:
# 특정 워크플로우의 초기는 다음과 같이 생긴다

from entity.tasks import GuardCondition, TaskSpec, TaskType, Layer, AgentNature, AgentRole
from entity.validators import TokenValidator, SpecChainValidator
from entity.tokens import Token
from core.utils import load_resource_specs


resource_db = load_resource_specs("./ResourceSpec/TokenSpec.yaml")
validator = TokenValidator(resource_db)

# Chain Validator 인스턴스 생성
chain_check = SpecChainValidator(validator)


# 최초 토큰 생성
valid_token = Token(
    trace_id = "dfhou3898dfalss28fhs", # 향후 랜덤 생성기 필요
    content = {
        "text" : "Samsung Elec Risk Analysis",
        "risk_score" : 0.96
    }
)

### 연관 태스크 정의
## Task는 Transition이다 (화살표다)
task_A = TaskSpec(
    # 일반 설명
    task_id = "TASK_TEST_001",
    description="금융 뉴스 감성 분석",
    type=TaskType.PYTHON_FUNC, 
    target="utils.analysis:calculate_sentiment", # 실행 함수 경로

    # config
    config = {
        'business_context' : 'ojs가 분석을 위해 만든 최초 example'
    },

    # 구조
    layer = Layer.OBSERVATION,
    required_agent_roles=[AgentRole.CONSULTANT],
    required_agent_types=[AgentNature.LLM],

    # 가드 구조 선언
    guards = [
        GuardCondition(
            target_topic_id = "TOPIC_FINANCE", 
            min_relevance=0.7,
            description="금융 관련성 0.7 이상 필수"
        )
    ],

    # TokenSpec.yaml 파일 참고
    input_spec_id="RS_RISK_TOKEN_V1",
    output_spec_id="RS_RISK_TOKEN_V2"
)

### 연관 태스크 정의
## Task는 Transition이다 (화살표다)
task_B = TaskSpec(
    # 일반 설명
    task_id = "TASK_TEST_002",
    description="분석 결과 추가 분석",
    type=TaskType.PYTHON_FUNC, 
    target="utils.analysis:calculate_sentiment", # 실행 함수 경로

    # config
    config = {
        'business_context' : 'ojs가 분석을 위해 만든 최초 example'
    },

    # 구조
    layer = Layer.OBSERVATION,
    required_agent_roles=[AgentRole.CONSULTANT],
    required_agent_types=[AgentNature.LLM],

    # 가드 구조 선언
    guards = [
        GuardCondition(
            target_topic_id = "TOPIC_FINANCE", 
            min_relevance=0.7,
            description="금융 관련성 0.7 이상 필수"
        )
    ],

    # TokenSpec.yaml 파일 참고
    input_spec_id="RS_RISK_TOKEN_V1",
    output_spec_id="RS_RISK_TOKEN_V2"
)

chain_check.validate_link(task_A, task_B)

### /process/process.py

In [ ]:
from typing import Dict, List, Deque, Any, Optional
from collections import deque, defaultdict
import logging
from pydantic import ValidationError
from core import logging_utils
from dataclasses import dataclass

@dataclass
class FiringResult:
    is_fired: bool
    new_token : Optional[Any] = None
    reason: str = "Success"
    execution_time: float = 0.0

class Process:
    """
    Graph Topology & Token State Container
    책임 : Task 간 연결 관리 및 현재 토큰 위치 추적
    """
    def __init__(self, process_id : str):

        # Registry & Graph
        self.process_id = process_id
        self.tasks : Dict[str, TaskSpec] = {} # {task_id : TaskSpec 객체}
        self.graph : Dict[str, List[str]] = defaultdict(list) # {source_id: [target_id_1, target_id_2]}. 신규생성 key는 무조건 list를 지님

        # Runtime State
        self.token_queue : Deque[tuple[str, Any]] = deque() # [(target_task_id, token_obj)]
        self.completed_tokens: List[Any] = [] # 처리 완료 토큰 보관소

        self.logger = logging_utils.get_logger(f"Proc_{process_id}")
        self.is_compiled = False # 컴파일 완료 여부 플래그

    def add_task(self, task : TaskSpec):
        # 프로세스 정의는 add_task와 add_link로 이행하는 것을 권고
        if task.task_id in self.tasks:
            self.logger.warning(f"Task ID {task.task_id} already exists. Overwriting")
        self.tasks[task.task_id] = task

    def add_link(self, source_id : str, target_id : str):
        """
        task간 연결 (특정 분기에 존재하는 리스트에 task를 추가하는것)
        source_id 실행 완료 -> target_id 실행 시도
        - 미등록 task 실행시 에러
        """
        if source_id not in self.tasks:
            raise ValueError(f"Source task '{source_id} not registered")
        if target_id not in self.tasks:
            raise ValueError(f"Target task '{target_id}' not registered")
        
        self.graph[source_id].append(target_id) # 리스트에 append
        self.is_compiled = False # 구조 변경시 재컴파일 필요
    
    def compile(self, chain_validator) -> bool:
        """
        Static Engine Phase
        엔진 가동 전 정의된 그래프의 무결성 검증. 

        검증항목
        1. Token Spec 호환성
        2. 구조적 데드락 검증 - Directed Acyclity 강제

        검증 모두 통과시 is_compiled=TRUE
        """
        # 1. Spec Compatibility 검증
        for source_id, target_ids in self.graph.items():
            source_task = self.tasks[source_id]
            for target_id in target_ids:
                target_task = self.tasks[target_id]
                if not chain_validator.validate_link(source_task, target_task):
                    self.logger.error(f"[SPEC CHAIN ERROR] {source_id} -> {target_id} mismatch")
                    error_count += 1
                
        # 2. 데드락 검증
        if self._detect_cycle():
            self.logger.error("[TOPOLOGY ERROR] 데드락 발생! ")
            error_count += 1

        if error_count > 0:
            self.logger.critical(f"[COMPILE FAILURE] : {error_count} errors.")
            return False
        
        # 검증 전부 통과시 True Return
        self.is_compiled = True
        self.logger.info(f"[COMPILE SUCCESS] Process Topology 정상.")
        return True
    
    def inject_token(self, start_task_id : str, token: Any):
        """
        [RUNTIME ENTRY POINT]
        초기 토큰을 특정 시작 task에 주입
        """
        if not self.is_compiled:
            self.logger.warning("[WARNING]: Process가 컴파일되지 않고 실행중입니다.")

        if start_task_id not in self.tasks:
            raise ValueError(f"Entry task {start_task_id}가 task registry에서 확인되지 않습니다. add_task를 해주세요")
        
        # 시작 TASK의 Input Spec과 토큰이 맞는지 체크는 Engine레벨에서 수행
        self.token_queue.append((start_task_id, token))
        self.logger.info(f"토큰 주입 = {start_task_id}")

    def get_next_nodes(self, current_task_id: str) -> List[TaskSpec]:
        """
        현재 Task 완료 후 이동 가능한 후보 Task 리스트 반환
        실행 엔진의 Rounting 단계에서 호출될 메서드
        """
        # graph 내에서 현재 노드의 다음 가능한 노드 리스트를 리턴
        next_ids = self.graph.get(current_task_id, [])
        
        # 실제 TaskSpec 객체 리스트 반환
        return [self.tasks[nid] for nid in next_ids]
    
    def get_task(self, task_id : str) -> Optional[TaskSpec]:
        " task id로 TaskSpec를 조회하는 메서드"
        return self.tasks.get(task_id)

    # (2. 데드락 검증)을 위한 검증 함수
    def _detect_cycle(self) -> bool:
        """
        DFS를 이용한 순환 참조(Cycle) 탐지
        순환 참조 존재시 True 리턴
        이외에 False 리턴
        """
        visited = set()
        recursion_stack = set()

        def dfs(node_id):   
            visited.add(node_id)
            recursion_stack.add(node_id)

            for neighbor_id in self.graph.get(node_id, []):
                if neighbor_id not in visited:
                    if dfs(neighbor_id):
                        return True
                elif neighbor_id in recursion_stack:
                    # 방문 중인 경로에 다시 도달 -> Cycle 발생
                    self.logger.error(f"Cycle detected at node: {neighbor_id}")
                    return True
            
            recursion_stack.remove(node_id)
            return False

        # 모든 노드에 대해 DFS 수행 (단절된 그래프 고려)
        for node_id in self.tasks:
            if node_id not in visited:
                if dfs(node_id):
                    return True
        return False

In [13]:
list_dict = defaultdict(list)

In [21]:
list_dict

defaultdict(list, {'key_1': 'a', 'key_2': []})

### /process/engine.py